In [1]:
using Pkg; Pkg.activate("../../../")

  Activating new project at `~/Code/Repos/Temp/CausalityTools.jl/src/methods/infomeasures`


In [ ]:
using Revise, CausalityTools

In [ ]:
using CairoMakie
using Distributions
using Statistics: quantile
usin
Rx = Exponential(0.7);
n = 500
x = rand(Rx, n)
y = x + randn(n)

function empcdf(x)
    N = length(x)
    F̂ = [count(xᵢ .<= x)  for xᵢ in x] / N
end

function inv_normal_copula_transform(x)
    ex = empcdf(x)
    t = zeros(length(ex))
    N = Normal(0, 1)
    for (i, eᵢ) in enumerate(ex)
        if eᵢ == 1.0 # Something weird happens when x = 1 or x = 0. Check why.
            t[i] = quantile(N, 1-eps())
        elseif eᵢ == 0.0
            t[i] = quantile(N, eps())
        else
            t[i] = quantile(N, eᵢ)
        end
    end

    return t
end

ex = empcdf(x)
ey = empcdf(y)
# Apply transformation (ok, as long as we don't change the ecdf). Use inverse normal cdf
#, ie.e quantile, so that we can use parametric estimator of MI.
X = inv_normal_copula_transform(x)
Y = inv_normal_copula_transform(y)

f = Figure(resolution = (600, 300))
ax = Axis(f[ 1, 1])
scatter!(ax, x, y)
ax2 = Axis(f[1, 2])
scatter!(ax2, X, Y)
xlims!(ax2, (-3 , 3))
ylims!(ax2, (-3, 3))
f

